In [60]:
import pandas as pd
import numpy as np

df = pd.read_csv('questions.csv')

for col in df.columns:
    df[col] = df[col].replace(',', '', regex=True)
    df[col] = df[col].replace('$', '', regex=True)

df['Sr. No'] = df['Sr. No'].fillna(method='ffill')
df['Sr. No'] = df['Sr. No'].astype(int)
grouped_df = df.groupby('Sr. No')
required_columns = ['Data'] + [f'Unnamed: {i}' for i in range(3, 10)]
processed_groups = {}
for name, group in grouped_df:
    missing_columns = [
        col for col in required_columns if col not in group.columns]
    if missing_columns:
        print(f"Missing columns in group {name}: {missing_columns}")
        continue
    filtered_group = group[required_columns]
    filtered_group.fillna('', inplace=True)
    processed_groups[name] = filtered_group
df.drop(columns=required_columns, inplace=True)
new_df = df.groupby('Sr. No').first()

for i in range(1, len(processed_groups) + 1):
    d = processed_groups[i]
    new_columns = d.iloc[0]
    d.columns = new_columns
    d = d.drop(d.index[0])
    d.reset_index(drop=True, inplace=True)
    d.replace("", float("NaN"), inplace=True)
    d.dropna(how='all', axis=1, inplace=True)
    d = d.drop(d.index[-1])
    processed_groups[i] = d

new_df = new_df.assign(Data=processed_groups.values())
df = new_df
df.reset_index(drop=True, inplace=True)
df.head()

C:\Users\Aditya\AppData\Local\Temp\ipykernel_14140\1602288140.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Sr. No'] = df['Sr. No'].fillna(method='ffill')
C:\Users\Aditya\AppData\Local\Temp\ipykernel_14140\1602288140.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  filtered_group.fillna('', inplace=True)
C:\Users\Aditya\AppData\Local\Temp\ipykernel_14140\1602288140.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_group.fillna('', inplace=True)
C:\Users\Aditya\AppData\Local\Temp\ipykernel_14140\1602288140.py:22: FutureWarning:

,Questions,Alpha,Type,Data
0,A researcher is studying the effect of differe...,0.05,CRD,0 Fertilizer A Fertilizer B Fertilizer C 0 ...
1,A company wants to compare customer satisfacti...,0.01,CRD,7 Model X Model Y Model Z 0 7 8 ...
2,To study the effect of three different diets o...,0.05,CRD,14 Diet A Diet B Diet C 0 5 4 ...
3,The preference scores (out of 10) for three di...,0.05,FD,21 Market Packaging A Packaging B Packaging...
4,The productivity (in units) for employees usin...,0.05,FD,29 Shift Tool A Tool B Tool C 0 Day...


In [61]:
temp = df.iloc[3]['Data']
temp

21,Market,Packaging A,Packaging B,Packaging C
0,Market 1,8,7,9
1,Market 1,9,6,8
2,Market 1,7,8,9
3,Market 2,6,9,7
4,Market 2,8,7,6
5,Market 2,7,8,9


In [62]:
def isFirstColumnMergedHeader(df):
    firstColumn = df.to_numpy()
    firstColumn = firstColumn.T
    firstColumn = list(set(firstColumn[0]))
    firstColumn = [x for x in firstColumn if not pd.isna(x)]
    return len(firstColumn) == 1

isFirstColumnMergedHeader(temp)

False

In [63]:
def isSecondColumnGroupedHeader(df):
    column = df.to_numpy()
    column = column.T
    column = [x for x in column[1] if not pd.isna(x)]
    sample = list(set(column))
    countOfSample = [column.count(x) for x in sample]
    if len(set(countOfSample)) == 1 and countOfSample[0] > 1:
        return True
    else:
        if any(str(x).isalpha() for x in sample):
            return True
        else:
            return False


isSecondColumnGroupedHeader(temp)

False

In [64]:
def isFirstColumnHeader(df):
    if isFirstColumnMergedHeader(df):
        return True
    firstColumn = df.to_numpy()
    firstColumn = firstColumn.T[0]
    if any(str(n).isalpha() for n in firstColumn):
        return True
    elif all(str(n).isnumeric() for n in firstColumn):
        firstColumn = [int(x) for x in firstColumn]
        if firstColumn == list(range(firstColumn[0], firstColumn[-1] + 1)):
            return True
        else:
            return False
    else:
        for x in firstColumn:
            try:
                float(x)
            except ValueError:
                return True
        return False

isFirstColumnHeader(temp)

True

In [65]:
def isFirstColumnGroupedHeader(df):
    if isFirstColumnHeader(df):
        column = df.to_numpy()
        column = column.T[0]
        column = [x for x in column if not pd.isna(x)]
        sample, countOfSample = np.unique(column, return_counts=True)
        if all(x == countOfSample[0] for x in countOfSample) and countOfSample[
                0] > 1:
            return True
        else:
            return False
    else:
        return False


isFirstColumnGroupedHeader(temp)

True

In [66]:
def isFirstRowHeader(df, data):
    firstRow = df.iloc[0]
    firstRow = list(firstRow)
    firstRow = [x for x in firstRow if not pd.isna(x)]
    if isFirstColumnHeader(data):
        firstRow = firstRow[1:]
    if any(n.isalpha() for n in firstRow):
        return True
    elif all(n.isnumeric() for n in firstRow):
        num_list = [int(n) for n in firstRow]
        if num_list == list(range(num_list[0], num_list[0] + len(num_list))):
            return True
        else:
            return False
    else:
        if all(isinstance(x, (int, float)) for x in firstRow):
            if all(x.is_integer() for x in firstRow):
                return False
            else:
                return True
        else:
            return False

In [67]:
def extractData(df):
    data = []
    if isFirstColumnMergedHeader(df):
        if isSecondColumnGroupedHeader(df):
            data = df.iloc[:, 2:]
        else:
            data = df.iloc[:, 1:]
    elif isFirstColumnHeader(df):
        data = df.iloc[:, 1:]
    elif isFirstColumnGroupedHeader(df):
        data = df.iloc[:, 1:]
    else:
        data = df.iloc[:, :]

    if isFirstRowHeader(data, df):
        data = data[1:]
    data.reset_index(drop=True, inplace=True)
    data = data.to_numpy()
    for i in range(len(data)):
        for j in range(len(data[i])):
            try:
                data[i][j] = float(data[i][j])
            except:
                pass
    return data

extractData(temp)


array([[8.0, 7.0, 9.0],
       [9.0, 6.0, 8.0],
       [7.0, 8.0, 9.0],
       [6.0, 9.0, 7.0],
       [8.0, 7.0, 6.0],
       [7.0, 8.0, 9.0]], dtype=object)

In [68]:
df['isFirstColumnMergedHeader'] = df['Data'].apply(isFirstColumnMergedHeader)
df['isSecondColumnGroupedHeader'] = df['Data'].apply(isSecondColumnGroupedHeader)
df['isFirstColumnHeader'] = df['Data'].apply(isFirstColumnHeader)
df['isFirstColumnGroupedHeader'] = df['Data'].apply(isFirstColumnGroupedHeader)
df['isFirstRowHeader'] = df.apply(lambda x: isFirstRowHeader(x['Data'], x['Data']), axis=1)
df['Extracted Data'] = df['Data'].apply(extractData)

df.head()

,Questions,Alpha,Type,Data,isFirstColumnMergedHeader,isSecondColumnGroupedHeader,isFirstColumnHeader,isFirstColumnGroupedHeader,isFirstRowHeader,Extracted Data
0,A researcher is studying the effect of differe...,0.05,CRD,0 Fertilizer A Fertilizer B Fertilizer C 0 ...,False,False,False,False,False,"[[45.0, 50.0, 55.0], [48.0, 52.0, 53.0], [46.0..."
1,A company wants to compare customer satisfacti...,0.01,CRD,7 Model X Model Y Model Z 0 7 8 ...,False,False,False,False,False,"[[7.0, 8.0, 6.0], [6.0, 7.0, 7.0], [8.0, 9.0, ..."
2,To study the effect of three different diets o...,0.05,CRD,14 Diet A Diet B Diet C 0 5 4 ...,False,False,False,False,False,"[[5.0, 4.0, 6.0], [6.0, 5.0, 7.0], [4.0, 6.0, ..."
3,The preference scores (out of 10) for three di...,0.05,FD,21 Market Packaging A Packaging B Packaging...,False,False,True,True,False,"[[8.0, 7.0, 9.0], [9.0, 6.0, 8.0], [7.0, 8.0, ..."
4,The productivity (in units) for employees usin...,0.05,FD,29 Shift Tool A Tool B Tool C 0 Day...,False,False,True,True,False,"[[20.0, 18.0, 22.0], [21.0, 19.0, 23.0], [19.0..."


In [69]:
df.to_csv('output.csv', index=False)

In [70]:
import pandas as pd

df = pd.read_csv('output.csv')

In [71]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

features = ['isFirstColumnMergedHeader', 'isSecondColumnGroupedHeader',
            'isFirstColumnHeader', 'isFirstColumnGroupedHeader', 'isFirstRowHeader']
X = df[features]
y = df['Type']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Support Vector Machine": SVC()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print(f"{name} Accuracy: {accuracy_score(y_test, predictions)}")

Logistic Regression Accuracy: 1.0
Decision Tree Accuracy: 1.0
Random Forest Accuracy: 1.0
Support Vector Machine Accuracy: 0.9047619047619048


In [72]:
from sklearn.model_selection import cross_val_score

for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=5)
    print(f"{name} Average Accuracy: {scores.mean()}")

Logistic Regression Average Accuracy: 0.9238095238095239
Decision Tree Average Accuracy: 0.9238095238095239


c:\Users\Aditya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\Aditya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\Aditya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Random Forest Average Accuracy: 0.9238095238095239
Support Vector Machine Average Accuracy: 0.9523809523809523


c:\Users\Aditya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


In [73]:
import joblib

for name, model in models.items():
    joblib.dump(model, f"saves/{name}.pkl")

In [74]:
sample_data = pd.read_csv('FDsamplequestion.csv')

sample_data

,Unnamed: 0,Unnamed: 1,Test Marks,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,A,B,C,D
1,Student,Aditya,10,12,11,14
2,NaN,Priyanka,9,3,6,12
3,NaN,Shivani,7,14,9,10
4,NaN,Mithil,11,15,12,9


In [75]:
characteristics = {
    "isFirstColumnMergedHeader": isFirstColumnMergedHeader(sample_data),
    "isSecondColumnGroupedHeader": isSecondColumnGroupedHeader(sample_data),
    "isFirstColumnHeader": isFirstColumnHeader(sample_data),
    "isFirstColumnGroupedHeader": isFirstColumnGroupedHeader(sample_data),
    "isFirstRowHeader": isFirstRowHeader(sample_data, sample_data),
}


print(characteristics)

{'isFirstColumnMergedHeader': True, 'isSecondColumnGroupedHeader': True, 'isFirstColumnHeader': True, 'isFirstColumnGroupedHeader': False, 'isFirstRowHeader': True}


In [76]:
import os
models_path = "saves/"

models = {}

for file in os.listdir(models_path):
    if file.endswith(".pkl"):
        model = joblib.load(models_path + file)
        models[file[:-4]] = model

models


{'Decision Tree': DecisionTreeClassifier(),
 'Logistic Regression': LogisticRegression(),
 'Random Forest': RandomForestClassifier(),
 'Support Vector Machine': SVC()}

In [77]:
predictions = {}
for name, model in models.items():
    predictions[name] = model.predict([list(characteristics.values())])[0]

predictions

c:\Users\Aditya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\Aditya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\Aditya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\Aditya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


{'Decision Tree': 'FD',
 'Logistic Regression': 'FD',
 'Random Forest': 'FD',
 'Support Vector Machine': 'FD'}